In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

/kaggle/input/4datathon/sample_submission.csv
/kaggle/input/4datathon/prepared_test.csv
/kaggle/input/4datathon/prepared_train.csv


In [ ]:
train_df = pd.read_csv('/kaggle/input/4datathon/preapared_train.csv')
test_df = pd.read_csv('/kaggle/input/4datathon/prepared_test.csv')

features = ['event_count', 'unique_products', 'unique_categories', 
            'event_type_ADD_CART', 'event_type_BUY', 'event_type_REMOVE_CART', 
            'event_type_VIEW', 'duration']

X_train = train_df[features]
y_train = train_df['session_value']

X_test = test_df[features]

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

predictions = model.predict(X_test)

In [ ]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

train_df = pd.read_csv('/kaggle/input/4datathon/prepared_train.csv')
test_df = pd.read_csv('/kaggle/input/4datathon/prepared_test.csv')

features = ['event_count', 'unique_products', 'unique_categories',
            'event_type_ADD_CART', 'event_type_BUY', 'event_type_REMOVE_CART',
            'event_type_VIEW','duration']

X_train = train_df[features]
y_train = train_df['session_value']
X_test = test_df[features]

# XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

# LightGBM 
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)

# CatBoost 
cat_model = CatBoostRegressor(n_estimators=100, learning_rate=0.1, verbose=0, random_state=42)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)

# --- TAHMİNLERİN ORTALAMASI

predictions_df = pd.DataFrame({
    'xgb': xgb_predictions,
    'lgb': lgb_predictions,
    'cat': cat_predictions
})

final_predictions = predictions_df.mean(axis=1)

# --- SUBMISSIONS

submission_df = pd.DataFrame({
    'user_session': test_df['user_session'],
    'session_value': final_predictions
})

submission_df.to_csv('ensembled_tahminler.csv', index=False)

print("Ensembled tahminler başarıyla 'ensembled_tahminler.csv' dosyasına kaydedildi.")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# ---- Load ----
train_df = pd.read_csv('/kaggle/input/4datathon/prepared_train.csv')
test_df  = pd.read_csv('/kaggle/input/4datathon/prepared_test.csv')

# ---- Features ----
features = ['unique_products', 'unique_categories',
            'event_type_ADD_CART', 'event_type_BUY', 'event_type_REMOVE_CART',
            'event_type_VIEW', 'duration']

X = train_df[features]
y = train_df['session_value']
X_test = test_df[features]

# ---- Log-target ----
y_log = np.log1p(y)

# ---- Train/Validation split ----
X_tr, X_val, y_tr, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

# ---- Model ----
model = xgb.XGBRegressor(
    n_estimators=5000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    reg_alpha=0.1,
    reg_lambda=1.0,
    objective='reg:squarederror',
    tree_method='hist',
    random_state=42
)

# ---- Fit with early stopping ----
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    early_stopping_rounds=200,
    verbose=False
)

# ---- Validation RMSE ----
val_pred_log = model.predict(X_val)
val_pred = np.expm1(val_pred_log)
val_rmse = mean_squared_error(np.expm1(y_val), val_pred, squared=False)
print("Validation RMSE (original scale):", round(val_rmse, 6))
print("Best iteration:", model.best_iteration)

# ---- Test predictions ----
test_pred = np.expm1(model.predict(X_test))

submission_df = pd.DataFrame({
    'user_session': test_df['user_session'],
    'session_value': test_pred
})
submission_df.to_csv('tahminler.csv', index=False)
print("Saved tahminler.csv")

In [ ]:
####BEST submission
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

train_df = pd.read_csv('/kaggle/input/4datathon/prepared_train.csv')

test_df = pd.read_csv('/kaggle/input/4datathon/prepared_test.csv')

features = ['unique_products', 'unique_categories',
            'event_type_ADD_CART', 'event_type_BUY', 'event_type_REMOVE_CART',
            'event_type_VIEW', 'duration']


X_train = train_df[features]
y_train = train_df['session_value']

X_test = test_df[features]

model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

submission_df = pd.DataFrame({

'user_session': test_df['user_session'],
'session_value': predictions})

submission_df.to_csv('tahminler.csv', index=False)
print("Tahminler başarıyla 'tahminler.csv' dosyasına kaydedildi.")

In [ ]:
# 1
# XGBoost + Optuna Hyperparam Tuning
!pip -q install optuna

import pandas as pd
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

# ---- Load prepared data ----
train = pd.read_csv("/kaggle/input/4datathon/prepared_train.csv")
test  = pd.read_csv("/kaggle/input/4datathon/prepared_test.csv")
sample = pd.read_csv("/kaggle/input/4datathon/sample_submission.csv")

# ---- Feature set ----
core_features = [
    "unique_products","unique_categories",
    "event_type_ADD_CART","event_type_BUY",
    "event_type_REMOVE_CART","event_type_VIEW",
    "duration"
]
time_features = core_features + ["hour","day_of_week","day_of_month"]

use_time = all(c in train.columns for c in time_features)
features = time_features if use_time else core_features
print("Using features:", features)

X = train[features]
y = train["session_value"].astype(float)
X_test = test[features]

# ---- KFold ----
KFOLDS = 5
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

# ---- XGBoost fit helper ----
def fit_xgb_dmatrix(Xtr, ytr, Xv, yv, params, num_boost_round=5000, early_stopping_rounds=200):
    dtrain = xgb.DMatrix(Xtr, label=ytr)
    dvalid = xgb.DMatrix(Xv,  label=yv)
    evals = [(dtrain, "train"), (dvalid, "valid")]
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=evals,
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=False
    )
    return booster

def xgb_predict(booster, X_):
    d_ = xgb.DMatrix(X_)
    ntree_limit = getattr(booster, "best_ntree_limit", 0)
    return booster.predict(d_, ntree_limit=ntree_limit) if ntree_limit else booster.predict(d_)

# ---- Optuna objective ----
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "eta": trial.suggest_float("eta", 0.01, 0.08, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1e-1, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 1e2, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "seed": 42,
    }

    rmses = []
    # Log-target CV
    y_log = np.log1p(y)
    for tr_idx, val_idx in kf.split(X, y):
        Xtr, Xv = X.iloc[tr_idx], X.iloc[val_idx]
        ytr, yv = y_log.iloc[tr_idx], y_log.iloc[val_idx]

        booster = fit_xgb_dmatrix(
            Xtr, ytr, Xv, yv,
            params=params,
            num_boost_round=5000,
            early_stopping_rounds=200
        )
        p_val = xgb_predict(booster, Xv)
        rmse = np.sqrt(mean_squared_error(np.expm1(yv), np.expm1(p_val)))
        rmses.append(rmse)

    cv_rmse = float(np.mean(rmses))
    return cv_rmse

# ---- Optuna Tuning ----
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize")
N_TRIALS = 50 
study.optimize(objective, n_trials=N_TRIALS)

print("Best CV RMSE:", study.best_value)
print("Best params:", study.best_params)

# ---- En iyi parametrelerle 5-Fold OOF + Test (LOG-TARGET) ----
best_params = study.best_params.copy()
best_params.update({
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "seed": 42
})

oof = np.zeros(len(X))
test_preds_accum = np.zeros(len(X_test))

y_log = np.log1p(y)
fold_best_rounds = []

for tr_idx, val_idx in kf.split(X, y):
    Xtr, Xv = X.iloc[tr_idx], X.iloc[val_idx]
    ytr, yv = y_log.iloc[tr_idx], y_log.iloc[val_idx]

    booster = fit_xgb_dmatrix(
        Xtr, ytr, Xv, yv,
        params=best_params,
        num_boost_round=10000,
        early_stopping_rounds=300
    )
    # Val
    p_val = xgb_predict(booster, Xv)
    oof[val_idx] = np.expm1(p_val)

    # Test
    p_test = xgb_predict(booster, X_test)
    test_preds_accum += np.expm1(p_test) / KFOLDS

    #diagnostic
    best_iter = getattr(booster, "best_iteration", None)
    if best_iter is None:
        best_iter = int(getattr(booster, "best_ntree_limit", 0) or 0)
    fold_best_rounds.append(best_iter)

# ---- OOF RMSE ----
oof_rmse = np.sqrt(mean_squared_error(y, oof))
print("OOF RMSE (log-target with best params):", round(oof_rmse, 6))
print("Fold best rounds:", fold_best_rounds)

# ---- LOG-TARGET submission ----
sub_log = sample.copy()
sub_log["session_value"] = test_preds_accum
sub_log.to_csv("submission_xgb_optuna_log.csv", index=False)
print("Saved: submission_xgb_optuna_log.csv")


oof_normal = np.zeros(len(X))
test_preds_normal = np.zeros(len(X_test))

for tr_idx, val_idx in kf.split(X, y):
    Xtr, Xv = X.iloc[tr_idx], X.iloc[val_idx]
    ytr, yv = y.iloc[tr_idx], y.iloc[val_idx]

    booster = fit_xgb_dmatrix(
        Xtr, ytr, Xv, yv,
        params=best_params,
        num_boost_round=10000,
        early_stopping_rounds=300
    )
    p_val = xgb_predict(booster, Xv)
    oof_normal[val_idx] = p_val

    p_test = xgb_predict(booster, X_test)
    test_preds_normal += p_test / KFOLDS

oof_rmse_normal = np.sqrt(mean_squared_error(y, oof_normal))
print("OOF RMSE (normal target, same params):", round(oof_rmse_normal, 6))

sub_normal = sample.copy()
sub_normal["session_value"] = test_preds_normal
sub_normal.to_csv("submission_xgb_optuna_normal.csv", index=False)
print("Saved: submission_xgb_optuna_normal.csv")

In [ ]:
#2
import optuna, lightgbm as lgb, numpy as np, pandas as pd, os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

os.environ["LGBM_DEVICE_TYPE"] = "cpu" 

train = pd.read_csv("/kaggle/input/4datathon/prepared_train.csv")
test  = pd.read_csv("/kaggle/input/4datathon/prepared_test.csv")
sample = pd.read_csv("/kaggle/input/4datathon/sample_submission.csv")

core = ["unique_products","unique_categories","event_type_ADD_CART","event_type_BUY",
        "event_type_REMOVE_CART","event_type_VIEW","duration"]
time = core + ["hour","day_of_week","day_of_month"]
features = time if all(c in train.columns for c in time) else core

X, y = train[features], train["session_value"].astype(float)
KFOLDS = 5
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

def objective_lgb(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.06, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 127),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 120),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
        "random_state": 42,
        "verbosity": -1,
    }

    y_log = np.log1p(y)
    rmses = []
    for tr_idx, va_idx in kf.split(X, y):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y_log.iloc[tr_idx], y_log.iloc[va_idx]

        dtr = lgb.Dataset(Xtr, label=ytr)
        dva = lgb.Dataset(Xva, label=yva, reference=dtr)

        model = lgb.train(
            params,
            dtr,
            num_boost_round=3000,
            valid_sets=[dtr, dva],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(0)
            ],
        )
        p = model.predict(Xva, num_iteration=model.best_iteration)
        rmse = np.sqrt(mean_squared_error(np.expm1(yva), np.expm1(p)))
        rmses.append(rmse)
    return float(np.mean(rmses))

optuna.logging.set_verbosity(optuna.logging.WARNING)
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0, interval_steps=1)
study_lgb = optuna.create_study(direction="minimize", pruner=pruner)
study_lgb.optimize(objective_lgb, n_trials=40)

print("Best CV RMSE (LGBM):", study_lgb.best_value)
print("Best params (LGBM):", study_lgb.best_params)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[685]	training's rmse: 0.441695	valid_1's rmse: 0.450392
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[496]	training's rmse: 0.444282	valid_1's rmse: 0.444631
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[681]	training's rmse: 0.441671	valid_1's rmse: 0.449839
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[564]	training's rmse: 0.442508	valid_1's rmse: 0.449516
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[590]	training's rmse: 0.443155	valid_1's rmse: 0.446393
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[230]	training's rmse: 0.440064	valid_1's rmse: 0.450364
Training until validation scores don't improve for 100 rounds
Early stopping

In [ ]:
#3
# CatBoost + Optuna Hyperparam Tuning 

!pip -q install optuna catboost

import numpy as np, pandas as pd, optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

train = pd.read_csv("/kaggle/input/4datathon/prepared_train.csv")
test  = pd.read_csv("/kaggle/input/4datathon/prepared_test.csv")
sample = pd.read_csv("/kaggle/input/4datathon/sample_submission.csv")

core = ["unique_products","unique_categories",
        "event_type_ADD_CART","event_type_BUY",
        "event_type_REMOVE_CART","event_type_VIEW",
        "duration"]
time = core + ["hour","day_of_week","day_of_month"]
features = time if all(c in train.columns for c in time) else core
print("Using features:", features)

X, y = train[features], train["session_value"].astype(float)
X_test = test[features]
y_log = np.log1p(y)

KFOLDS = 5
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

def objective_cb(trial):
    params = {
        "loss_function": "RMSE",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.08, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 20.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 5.0),
        "random_strength": trial.suggest_float("random_strength", 0.5, 5.0, log=True),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "verbose": False,
        "task_type": "GPU" if False else "CPU",
        # Early stopping
        "od_type": "Iter",
        "od_wait": 200
    }
    rmses = []
    for tr_idx, va_idx in kf.split(X, y):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y_log.iloc[tr_idx], y_log.iloc[va_idx]

        model = CatBoostRegressor(
            **params,
            iterations=5000
        )
        model.fit(Xtr, ytr, eval_set=(Xva, yva), verbose=False)
        p = model.predict(Xva)
        rmse = np.sqrt(mean_squared_error(np.expm1(yva), np.expm1(p)))
        rmses.append(rmse)

    return float(np.mean(rmses))

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_cb = optuna.create_study(direction="minimize",
                               pruner=optuna.pruners.MedianPruner(n_startup_trials=5, interval_steps=1))
study_cb.optimize(objective_cb, n_trials=40)

print("Best CV RMSE (CatBoost):", study_cb.best_value)
print("Best params (CatBoost):", study_cb.best_params)

Using features: ['unique_products', 'unique_categories', 'event_type_ADD_CART', 'event_type_BUY', 'event_type_REMOVE_CART', 'event_type_VIEW', 'duration', 'hour', 'day_of_week', 'day_of_month']
Best CV RMSE (CatBoost): 19.075487209144228
Best params (CatBoost): {'learning_rate': 0.02391012381358478, 'depth': 4, 'l2_leaf_reg': 1.016026708301995, 'bagging_temperature': 1.5559082067444363, 'random_strength': 3.130981007010143, 'border_count': 97}


In [ ]:

#4

import numpy as np, pandas as pd, optuna, xgboost as xgb, lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

train = pd.read_csv("/kaggle/input/4datathon/prepared_train.csv")
test  = pd.read_csv("/kaggle/input/4datathon/prepared_test.csv")
sample = pd.read_csv("/kaggle/input/4datathon/sample_submission.csv")

core = ["unique_products","unique_categories",
        "event_type_ADD_CART","event_type_BUY",
        "event_type_REMOVE_CART","event_type_VIEW",
        "duration"]
time = core + ["hour","day_of_week","day_of_month"]
features = time if all(c in train.columns for c in time) else core
X, y = train[features], train["session_value"].astype(float)
X_test = test[features]
y_log = np.log1p(y)

KFOLDS = 5
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

#XGBoost best params
xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "eta": 0.055091126943701545,
    "max_depth": 6,
    "min_child_weight": 1.737491912946531,
    "subsample": 0.7175975747321702,
    "colsample_bytree": 0.819515614621276,
    "reg_alpha": 2.2870536596663814e-07,
    "reg_lambda": 4.347571964357222,
    "gamma": 0.25383082245869804,
    "seed": 42
}

def fit_xgb(Xtr, ytr, Xva, yva, params, num_boost_round=10000, early_stopping_rounds=300):
    dtr = xgb.DMatrix(Xtr, label=ytr)
    dva = xgb.DMatrix(Xva, label=yva)
    booster = xgb.train(params, dtr, num_boost_round=num_boost_round,
                        evals=[(dtr,"train"),(dva,"valid")],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval=False)
    return booster

def xgb_predict(booster, X_):
    d_ = xgb.DMatrix(X_)
    ntree_limit = getattr(booster, "best_ntree_limit", 0)
    return booster.predict(d_, ntree_limit=ntree_limit) if ntree_limit else booster.predict(d_)

#LightGBM best params
lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.044178121043156375,
    "num_leaves": 67,
    "max_depth": 4,
    "min_data_in_leaf": 27,
    "feature_fraction": 0.8343568042100146,
    "bagging_fraction": 0.9473869354783899,
    "bagging_freq": 2,
    "lambda_l1": 0.10457742075166329,
    "lambda_l2": 0.0004951102145322123,
    "random_state": 42,
    "verbosity": -1
}

#CatBoost best params
cb_best = {
    "learning_rate": 0.02391012381358478,
    "depth": 4,
    "l2_leaf_reg": 1.016026708301995,
    "bagging_temperature": 1.5559082067444363,
    "random_strength": 3.130981007010143,
    "border_count": 97
}

#Fold OOF + Test
oof_xgb = np.zeros(len(X)); test_xgb = np.zeros(len(X_test))
oof_lgb = np.zeros(len(X)); test_lgb = np.zeros(len(X_test))
oof_cb  = np.zeros(len(X)); test_cb  = np.zeros(len(X_test))

for tr_idx, va_idx in kf.split(X, y):
    Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
    ytr, yva = y_log.iloc[tr_idx], y_log.iloc[va_idx]

    # XGB
    booster = fit_xgb(Xtr, ytr, Xva, yva, xgb_params)
    p_va = xgb_predict(booster, Xva)
    p_te = xgb_predict(booster, X_test)
    oof_xgb[va_idx] = np.expm1(p_va); test_xgb += np.expm1(p_te)/KFOLDS

    # LGBM
    dtr = lgb.Dataset(Xtr, label=ytr); dva = lgb.Dataset(Xva, label=yva, reference=dtr)
    mdl_lgb = lgb.train(lgb_params, dtr, num_boost_round=10000,
                        valid_sets=[dtr, dva],
                        callbacks=[lgb.early_stopping(300), lgb.log_evaluation(0)])
    p_va = mdl_lgb.predict(Xva, num_iteration=mdl_lgb.best_iteration)
    p_te = mdl_lgb.predict(X_test, num_iteration=mdl_lgb.best_iteration)
    oof_lgb[va_idx] = np.expm1(p_va); test_lgb += np.expm1(p_te)/KFOLDS

    # CatBoost
    mdl_cb = CatBoostRegressor(
        loss_function="RMSE", random_seed=42, verbose=False,
        iterations=10000, od_type="Iter", od_wait=300, use_best_model=True,
        learning_rate=cb_best["learning_rate"],
        depth=cb_best["depth"],
        l2_leaf_reg=cb_best["l2_leaf_reg"],
        bagging_temperature=cb_best["bagging_temperature"],
        random_strength=cb_best["random_strength"],
        border_count=cb_best["border_count"],
        task_type="CPU"
    )
    mdl_cb.fit(Xtr, ytr, eval_set=(Xva, yva), verbose=False)
    p_va = mdl_cb.predict(Xva); p_te = mdl_cb.predict(X_test)
    oof_cb[va_idx] = np.expm1(p_va); test_cb += np.expm1(p_te)/KFOLDS

# OOF RMSEler
rmse_xgb = np.sqrt(mean_squared_error(y, oof_xgb))
rmse_lgb = np.sqrt(mean_squared_error(y, oof_lgb))
rmse_cb  = np.sqrt(mean_squared_error(y, oof_cb))
print("OOF RMSEs:", {"XGB":rmse_xgb, "LGBM":rmse_lgb, "CatBoost":rmse_cb})

def blend_objective(trial):
    w1 = trial.suggest_float("w_xgb", 0.0, 1.0)
    w2 = trial.suggest_float("w_lgb", 0.0, 1.0)
    w3 = trial.suggest_float("w_cb",  0.0, 1.0)
    s  = w1 + w2 + w3 + 1e-12
    pred = (w1*oof_xgb + w2*oof_lgb + w3*oof_cb)/s
    return float(np.sqrt(mean_squared_error(y, pred)))

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_blend = optuna.create_study(direction="minimize")
study_blend.optimize(blend_objective, n_trials=200)
print("Best OOF RMSE (blend):", study_blend.best_value)
print("Best weights:", study_blend.best_params)

# Final blended submission 
w = study_blend.best_params
ws = w["w_xgb"] + w["w_lgb"] + w["w_cb"] + 1e-12
test_blend = (w["w_xgb"]*test_xgb + w["w_lgb"]*test_lgb + w["w_cb"]*test_cb)/ws

sub = sample.copy()
sub["session_value"] = test_blend
sub.to_csv("submission_blended.csv", index=False)
print("Saved: submission_blended.csv")

pd.DataFrame({"session_value": test_xgb}, index=sample.index).to_csv("submission_xgb_only.csv", index=False)
pd.DataFrame({"session_value": test_lgb}, index=sample.index).to_csv("submission_lgb_only.csv", index=False)
pd.DataFrame({"session_value": test_cb},  index=sample.index).to_csv("submission_cb_only.csv",  index=False)
print("Saved: individual submissions (xgb/lgb/cb)")

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[574]	training's rmse: 0.443033	valid_1's rmse: 0.450155
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[535]	training's rmse: 0.444581	valid_1's rmse: 0.444565
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[577]	training's rmse: 0.443077	valid_1's rmse: 0.449743
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[729]	training's rmse: 0.442346	valid_1's rmse: 0.449385
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[675]	training's rmse: 0.443421	valid_1's rmse: 0.446395
OOF RMSEs: {'XGB': 19.163895601382755, 'LGBM': 19.555256299610907, 'CatBoost': 19.20897754710921}
Best OOF RMSE (blend): 19.08722805556598
Best weights: {'w_xgb': 0.4920647886263625, 'w_lgb': 0.00013259547077216311, 'w_cb': 0.3803